In [1]:
import pickle 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
# from gensim import corpora, models, similarities, matutils
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import pickle 

from sklearn.decomposition import NMF

# from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from collections import Counter

import tqdm
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive


#Data pre-processing 
load files

tokenization

build vocabularies

In [6]:
with open('fr_text_preprocessed_50000.txt', encoding='utf-8') as file1:
    articles = file1.readlines()
with open('en_sum_preprocessed_50000.txt', encoding='utf-8') as file2:
    sums = file2.readlines()


In [7]:
for i in range(5):
    print("summary #",i+1)
    print(articles[i])
    print(sums[i])
    print()

summary # 1
jean-jacques schuhl gilles leroy christian gailly yasmina khadra james ellroy amos naipaul rentree janvier sannonce sous meilleurs auspices force constater 491 romans contre 558 2009 tendance baisse enregistree lautomne saccentue principale victime cette glaciation litterature etrangere enregistre recul 167 romans contre 211 lan dernier plus bas niveau depuis 2001 peut-etre doit-on voir contrecoup laugmentation droits dauteur traduction aussi fait organisateurs salon livre choisi celebrer trente ans manifestation invitant non pays ecrivains francais etrangers litterature francaise cote marque leger flechissement 324 livres contre 347 lan passe premiers romans apres automne demi-teinte repartent hausse titres contre 2009 loin leffervescence septembre tension prix cette rentree hivernale placee sous signe decrue nen offre moins programme belle qualite ainsi chez gallimard retour jean-jacques schuhl entree fantomes dix ans apres goncourt ingrid caven deja annonce comme lun eve

In [50]:
# Setting the number of training sentences used to train
training_examples = len(articles)

en_words = Counter()
fr_words = Counter()
en_inputs = []
fr_inputs = []

# Tokenizing the English and French sentences and creating our word banks for both languages
for i in tqdm_notebook(range(training_examples)):
    en_tokens = word_tokenize(sums[i])
    fr_tokens = word_tokenize(articles[i])
    if len(en_tokens)==0 or len(fr_tokens)==0:
        continue
    for token in en_tokens:
        en_words.update([token.lower()])
    en_inputs.append([token.lower() for token in en_tokens] + ['_EOS'])
    for token in fr_tokens:
        fr_words.update([token.lower()])
    fr_inputs.append([token.lower() for token in fr_tokens] + ['_EOS'])

#Assigning an index to each word token, including the Start Of String(SOS), End Of String(EOS) and Unknown(UNK) tokens
en_words = ['_SOS','_EOS','_UNK'] + sorted(en_words,key=en_words.get,reverse=True)
en_w2i = {o:i for i,o in enumerate(en_words)}
en_i2w = {i:o for i,o in enumerate(en_words)}
fr_words = ['_SOS','_EOS','_UNK'] + sorted(fr_words,key=fr_words.get,reverse=True)
fr_w2i = {o:i for i,o in enumerate(fr_words)}
fr_i2w = {i:o for i,o in enumerate(fr_words)}

#Converting our English and French sentences to their token indexes
for i in range(len(en_inputs)):
    en_sentence = en_inputs[i]
    fr_sentence = fr_inputs[i]
    en_inputs[i] = [en_w2i[word] for word in en_sentence]
    fr_inputs[i] = [fr_w2i[word] for word in fr_sentence]


<ipython-input-50-14b74f61c9e7>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(training_examples)):


  0%|          | 0/50000 [00:00<?, ?it/s]

In [54]:
#store english and french vocabularies 
with open('en_word2index.pkl', 'wb') as f:
    pickle.dump(en_w2i, f)
        
with open('en_index2word.pkl', 'wb') as f:
    pickle.dump(en_i2w, f)

with open('fr_word2index.pkl', 'wb') as f:
    pickle.dump(fr_w2i, f)

with open('fr_index2word.pkl', 'wb') as f:
    pickle.dump(fr_i2w, f)


#Encoder

In [19]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, drop_prob=0):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=drop_prob, batch_first=True)

    def forward(self, inputs, hidden):
        #Embed input words
        embedded = self.embedding(inputs)
        #Pass the embedded word vectors into LSTM and return all outputs
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, device=device),
                torch.zeros(self.n_layers, batch_size, self.hidden_size, device=device))


#Decoder

In [20]:
class LuongDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, attention, n_layers=1, drop_prob=0.1):
        super(LuongDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.drop_prob = drop_prob

        #Our Attention Mechanism is defined in a separate class
        self.attention = attention

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.dropout = nn.Dropout(self.drop_prob)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.classifier = nn.Linear(self.hidden_size*2, self.output_size)
    
    def forward(self, inputs, hidden, encoder_outputs):
        #Embed input words
        embedded = self.embedding(inputs).view(1,1,-1)
        embedded = self.dropout(embedded)

        #Passing previous output word (embedded) and hidden state into LSTM cell
        lstm_out, hidden = self.lstm(embedded, hidden)

        #Calculating Alignment Scores - see Attention class for the forward pass function
        alignment_scores = self.attention(hidden[0], encoder_outputs)
        #Softmaxing alignment scores to obtain Attention weights
        attn_weights = F.softmax(alignment_scores.view(1,-1), dim=1)

        #Multiplying Attention weights with encoder outputs to get context vector
        context_vector = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs)

        #Concatenating output from LSTM with context vector
        output = torch.cat((lstm_out, context_vector),-1)
        #Pass concatenated vector through Linear layer acting as a Classifier
        output = F.log_softmax(self.classifier(output[0]), dim=1)
        return output, hidden, attn_weights

In [21]:
class Attention(nn.Module):
    def __init__(self, hidden_size, method="dot"):
        super(Attention, self).__init__()
        self.method = method
        self.hidden_size = hidden_size

        #Defining the layers/weights required depending on alignment scoring method
        if method == "general":
            self.fc = nn.Linear(hidden_size, hidden_size, bias=False)

        elif method == "concat":
            self.fc = nn.Linear(hidden_size, hidden_size, bias=False)
            self.weight = nn.Parameter(torch.FloatTensor(1, hidden_size))
  
    def forward(self, decoder_hidden, encoder_outputs):
        if self.method == "dot":
          #For the dot scoring method, no weights or linear layers are involved
          return encoder_outputs.bmm(decoder_hidden.view(1,-1,1)).squeeze(-1)
    
        elif self.method == "general":
            #For general scoring, decoder hidden state is passed through linear layers to introduce a weight matrix
            out = self.fc(decoder_hidden)
            return encoder_outputs.bmm(out.view(1,-1,1)).squeeze(-1)

        elif self.method == "concat":
            #For concat scoring, decoder hidden state and encoder outputs are concatenated first
            out = torch.tanh(self.fc(decoder_hidden+encoder_outputs))
            return out.bmm(self.weight.unsqueeze(-1)).squeeze(-1)

#Training

In [ ]:
hidden_size = 512
encoder = EncoderLSTM(len(fr_words), hidden_size).to(device)
attn = Attention(hidden_size,"concat")
decoder = LuongDecoder(hidden_size,len(en_words),attn).to(device)

lr = 0.001
encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr)

In [ ]:
EPOCHS = 20
teacher_forcing_prob = 0.5
encoder.train()
decoder.train()

for epoch in range(1, EPOCHS+1):
    avg_loss = 0.
    correct = 0.
    total = 0.
    
    with tqdm(fr_inputs, desc=f"Epoch {epoch}/{EPOCHS}") as pbar:
        for i, sentence in enumerate(pbar):
            loss = 0.
            h = encoder.init_hidden()
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            inp = torch.tensor(sentence).unsqueeze(0).to(device)
            encoder_outputs, h = encoder(inp, h)

            # First decoder input will be the SOS token
            decoder_input = torch.tensor([en_w2i['_SOS']], device=device)
            # First decoder hidden state will be last encoder hidden state
            decoder_hidden = h
            output = []
            teacher_forcing = True if random.random() < teacher_forcing_prob else False

            for ii in range(len(en_inputs[i])):
                decoder_output, decoder_hidden, attn_weights = decoder(decoder_input, decoder_hidden, encoder_outputs)
                # Get the index value of the word with the highest score from the decoder output
                top_value, top_index = decoder_output.topk(1)
                if teacher_forcing:
                    decoder_input = torch.tensor([en_inputs[i][ii]], device=device)
                else:
                    decoder_input = torch.tensor([top_index.item()], device=device)
                output.append(top_index.item())
                # Calculate the loss of the prediction against the actual word
                loss += F.nll_loss(decoder_output.view(1, -1), torch.tensor([en_inputs[i][ii]], device=device))

                # Calculate the number of correct predictions
                if top_index.item() == en_inputs[i][ii]:
                    correct += 1
                total += 1
            # print(output)
            # print(fr_inputs[i])

            loss = loss / len(en_inputs[i])
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            avg_loss += loss.item() / len(en_inputs)
            
            pbar.set_postfix(loss=loss.item(), accuracy=correct / total)
        
    accuracy = correct / total
    print(f"Epoch {epoch} loss: {avg_loss:.4f} | accuracy: {accuracy:.4f}")
    
    # Save model after every epoch
    torch.save({"encoder": encoder.state_dict(), "decoder": decoder.state_dict(), "e_optimizer": encoder_optimizer.state_dict(), "d_optimizer": decoder_optimizer.state_dict(), "epoch": epoch}, f"model_epoch_{epoch}.pth")


#Testing

In [9]:
#load dictionaries
with open('en_word2index.pkl', 'rb') as f:
    loaded_en_w2i = pickle.load(f)
with open('en_index2word.pkl', 'rb') as f:
    loaded_en_i2w = pickle.load(f)

with open('fr_word2index.pkl', 'rb') as f:
    loaded_fr_w2i = pickle.load(f)
with open('fr_index2word.pkl', 'rb') as f:
    loaded_fr_i2w = pickle.load(f)

with open('fr_text_preprocessed_test1000.txt', encoding='utf-8') as file3:
    test_articles = file3.readlines()




In [41]:
# en_inputs = []
fr_inputs = []

#Converting French testing articles to their token indexes
for i in range(len(test_articles)):

    #tokenize first:
    fr_tokens = word_tokenize(test_articles[i])
    fr_inputs.append([token.lower() for token in fr_tokens] + ['_EOS'])
    
    #word to index
    fr_article= fr_inputs[i]
    fr_inputs[i] = [loaded_fr_w2i[word] if word in loaded_fr_w2i else loaded_fr_w2i['_UNK'] for word in fr_article]




In [46]:
print("There are ",len(fr_inputs), "testing samples")
print("Visualize one article for testing: " + " ".join([loaded_fr_i2w[x] for x in fr_inputs[10]]))

There are  1000 testing samples
Visualize one article for testing: separation pouvoirs notion plus celebree nen guere daussi confuse 1789 declaration droits lhomme citoyen notait article quune societe laquelle separation pouvoirs netait garantie point constitution deux siecles plus tard quasi-totalite constitutions adoptees pays lex-bloc sovietique reprenaient presque mot mot meme enonce faits nul pourtant jamais envisage trois pouvoirs executif judiciaire legislatif puisque cest deux quil sagissait puissent fonctionner facon vraiment autonome derriere terme separation cest fait plutot lidee dun equilibre dune balance pouvoirs lon envisagee paradoxe regimes moins democratiques entendu sappuyer xixe siecle lexpression separation pouvoirs _UNK exemple lindependance pouvoir executif mettre labri toute velleite controle parlementaire cela ete cas second empire france concentrons-nous present posant deux questions avons-nous besoin dune separation dun equilibre pouvoirs oui selon quelles mo

In [104]:
# Define the path to the saved model
model_path = "model_epoch_5.pth"

# Load the saved model
checkpoint = torch.load(model_path)

# Extract the encoder and decoder state dictionaries from the loaded checkpoint
encoder_state_dict = checkpoint["encoder"]
decoder_state_dict = checkpoint["decoder"]

# Create the encoder and decoder models with the same architecture as the saved models
encoder = EncoderLSTM(len(loaded_fr_i2w), hidden_size).to(device)
# attn = Attention(hidden_size,"concat")
decoder = LuongDecoder(hidden_size,len(loaded_en_i2w),attn).to(device)

# Load the state dictionaries into the models
encoder.load_state_dict(encoder_state_dict)
decoder.load_state_dict(decoder_state_dict)

# Set the models to evaluation mode
encoder.eval()
decoder.eval()


LuongDecoder(
  (attention): Attention(
    (fc): Linear(in_features=512, out_features=512, bias=False)
  )
  (embedding): Embedding(45942, 512)
  (dropout): Dropout(p=0.1, inplace=False)
  (lstm): LSTM(512, 512)
  (classifier): Linear(in_features=1024, out_features=45942, bias=True)
)

In [ ]:
with open("en_predicted_sum_test1000.txt", "w") as f:
  for idx in range(0,len(fr_inputs)):
      h = encoder.init_hidden()
      inp = torch.tensor(fr_inputs[idx]).unsqueeze(0).to(device)
      encoder_outputs, h = encoder(inp,h)

      decoder_input = torch.tensor([loaded_en_w2i['_SOS']],device=device)
      decoder_hidden = h
      output = []
      attentions = []
      cnt = 0
      while True:
          decoder_output, decoder_hidden, attn_weights = decoder(decoder_input, decoder_hidden, encoder_outputs)
          _, top_index = decoder_output.topk(2)
          decoder_input = torch.tensor([top_index[0][1].item()],device=device)
          #If the decoder output is the End Of Sentence token, stop decoding process
          if top_index[0][1].item() == loaded_fr_w2i["_EOS"] or cnt >=25:
              break
          output.append(top_index[0][1].item())
          attentions.append(attn_weights.squeeze().cpu().detach().numpy())
          cnt+=1

      summary = ' '.join([loaded_en_i2w[x] for x in output])
      print(summary)
      f.write(summary + "\n")


In [114]:
with open('en_predicted_test1000_lstm.txt', 'r') as file:
    lines = file.readlines()
    lines = [line.strip() for line in lines]  # Remove trailing newline characters
print(lines[0])

in paris of a simple term are now department , is a simple term , where he was the victim . the government . .
